In [10]:
import pandas as pd
import tensorflow as tf
from scikeras.wrappers import KerasClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, roc_curve, precision_recall_curve, auc
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
import warnings

In [2]:
#Read initial data
file_path='rtfDataSet.csv'
df = pd.read_csv(file_path)
print(df.head())

       0      1      2      3      4      5      6      7      8      9  ...  \
0 -0.536  0.292 -0.684  0.123 -0.118  0.346 -0.308 -0.113  0.401 -0.399  ...   
1 -0.496 -0.298 -0.529  0.073 -0.072  0.107 -0.316 -0.066 -0.168 -0.099  ...   
2  0.008 -0.031 -0.240  0.178 -0.120  0.317 -0.481  0.031 -0.077  0.063  ...   
3 -0.188 -0.180 -0.062 -0.104 -0.136 -0.061 -0.216 -0.143  0.068 -0.189  ...   
4  0.038  0.155 -0.203 -0.088 -0.084 -0.164 -0.145 -0.168  0.008  0.143  ...   

      61     62     63     64     65     66     67     68     69  label  
0 -0.135 -0.063 -0.410 -0.223 -0.599 -0.136 -0.329 -0.132 -0.266      0  
1  0.189  0.007 -0.362 -0.151 -0.338 -0.031 -0.159 -0.097 -0.131      0  
2 -0.035  0.101 -0.098 -0.172 -0.290  0.033 -0.342 -0.321 -0.238      0  
3  0.146 -0.145 -0.527 -0.292 -0.663 -0.078 -0.194 -0.151 -0.268      0  
4  0.019 -0.084  0.164 -0.165 -0.126 -0.112  0.029 -0.138  0.148      1  

[5 rows x 71 columns]


In [12]:
#Reduce to sets and normalized
set_2=[2, 5, 9, 10, 16, 21, 28, 34, 38, 48, 49, 55, 61, 62, 64]
set_3=[1, 2, 7, 8, 9, 10, 19, 23, 28, 34, 35, 36, 42, 44, 45, 47, 48, 49, 53, 55, 56, 57, 58, 63, 64, 65, 67]
set_4=[0,1,2,3,6,9,12,14,15,16,17,19,22,23,26,27,29,31,33,34,35,38,41,44,45,47,50,51,54,55,57,58,65,66,67]

reduced_df_2=df.iloc[:,set_2]
reduced_df_3=df.iloc[:,set_3]
reduced_df_4=df.iloc[:,set_4]
y = df.iloc[:, -1]

scaler = MinMaxScaler()

reduced_df_2 = scaler.fit_transform(reduced_df_2)
reduced_df_3 = scaler.fit_transform(reduced_df_3)
reduced_df_4 = scaler.fit_transform(reduced_df_4)


#Array of sets
reduced_array=[reduced_df_2,reduced_df_3,reduced_df_4]

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(reduced_df_2, y, test_size=0.2, random_state=42)
X_train2, X_val2, y_train2, y_val2 = train_test_split(reduced_df_3, y, test_size=0.2, random_state=42)
#X_train2, X_val2, y_train2, y_val2 = train_test_split(X_amigo, y, test_size=0.2, random_state=42)

### Nuevo Gridsearch

In [ ]:
from tensorflow import keras
from kerastuner.tuners import RandomSearch
from kerastuner import HyperParameters

# Define the search space for hyperparameters
hp = HyperParameters()
hp.Choice('learning_rate', values=[0.1, 0.3, 0.5])
hp.Int('neurons', min_value=5, max_value=20, step=5)
hp.Choice('activation', values=['relu', 'sigmoid', 'linear'])
hp.Int('num_layers', min_value=1, max_value=5)  # Hyperparameter for the number of layers

# Define your input shape based on your data
input_shape = (X_train.shape[1],)

# Create a function to build the Keras model
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))

    for _ in range(hp.get('num_layers')):
        model.add(keras.layers.Dense(
            units=hp.Int('neurons', min_value=5, max_value=20, step=5),
            activation=hp.Choice('activation', ['relu', 'sigmoid', 'linear']), 
            kernel_regularizer=keras.regularizers.l1_l2(
                l1=hp.Choice('l1', [0.001, 0.0001, 0.005, 0.0045]),  # L1 regularization
                l2=hp.Choice('l2', [0.001, 0.0001, 0.005, 0.0045])  # L2 regularization
        )))
      

    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.005, 0.1, 0.3, 0.5])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize the tuner (Random Search in this case)
tuner = RandomSearch(
    build_model, objective='val_accuracy', max_trials=70, hyperparameters=hp)

# Perform the hyperparameter search
tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Get the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

# Print the best hyperparameters
print("Best Hyperparameters:", best_hyperparameters)

# Train the best model with the entire training dataset
best_model.fit(X_train, y_train, epochs=10, batch_size=32)

In [5]:
mlp = MLPClassifier()

# Define the parameter grid to search
param_grid = {
    'hidden_layer_sizes': [(5), (5,5), (5,5,5), (5,5,5,5),(5,5,5,5,5),
                           (10), (10,10), (10,10,10), (10,10,10,10),(10,10,10,10,10),
                           (15), (15,15), (15,15,15), (15,15,15,15),(15,15,15,15,15),
                           (20), (20,20), (20,20,20), (20,20,20,20),(20,20,20,20,20),
                           ],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'alpha': [0.1,0.001, 0.0001],
    'beta_1':[0.1,0.2, 0.5,0.6],
    'beta_2':[0.1,0.2, 0.5,0.6],
}

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(mlp, param_grid, cv=10, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Test Accuracy:", accuracy)

Best Hyperparameters: {'activation': 'relu', 'alpha': 0.1, 'beta_1': 0.5, 'beta_2': 0.5, 'hidden_layer_sizes': (15, 15)}
Test Accuracy: 0.7


c:\Users\charl\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## Model 1

In [16]:
warnings.filterwarnings("ignore")
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
counter = 1
epochs = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
learning_rate_now=0.1

# Neural Network
for X in reduced_array:
    print(f"Set {counter}:")
    for epoch in epochs:
        print(f"Epoch: {epoch}, Learning Rate: {learning_rate_now}")
        array_accuracy_epoch = []
        for fold, (train_index, val_index) in enumerate(stratified_kfold.split(X, y)):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]
            mlp = MLPClassifier(
                hidden_layer_sizes=(15, 15),
                activation='relu',
                max_iter=epoch,
                random_state=42,
                learning_rate_init=learning_rate_now,
                alpha=0.1,
                beta_1=0.5,
                beta_2=0.5
            )
            mlp.out_activation_ = 'sigmoid'
            mlp.fit(X_train, y_train)
            y_val_pred = mlp.predict(X_val)
            # Evaluate the model's performance on the validation set
            accuracy_val = accuracy_score(y_val, y_val_pred)
            array_accuracy_epoch.append(accuracy_val)
        # Calculate and store the average accuracy across all folds
        avg_accuracy_epoch = np.max(array_accuracy_epoch)
        print(f"Epoch: {epoch} Accuracy: {avg_accuracy_epoch}")

    counter += 1





Set 1:
Epoch: 1, Learning Rate: 0.1
Epoch: 1 Accuracy: 0.55
Epoch: 50, Learning Rate: 0.1
Epoch: 50 Accuracy: 0.825
Epoch: 100, Learning Rate: 0.1
Epoch: 100 Accuracy: 0.8
Epoch: 150, Learning Rate: 0.1
Epoch: 150 Accuracy: 0.8
Epoch: 200, Learning Rate: 0.1
Epoch: 200 Accuracy: 0.8
Epoch: 250, Learning Rate: 0.1
Epoch: 250 Accuracy: 0.8
Epoch: 300, Learning Rate: 0.1
Epoch: 300 Accuracy: 0.8
Epoch: 350, Learning Rate: 0.1
Epoch: 350 Accuracy: 0.8
Epoch: 400, Learning Rate: 0.1
Epoch: 400 Accuracy: 0.8
Epoch: 450, Learning Rate: 0.1
Epoch: 450 Accuracy: 0.8
Epoch: 500, Learning Rate: 0.1
Epoch: 500 Accuracy: 0.8
Set 2:
Epoch: 1, Learning Rate: 0.1
Epoch: 1 Accuracy: 0.55
Epoch: 50, Learning Rate: 0.1
Epoch: 50 Accuracy: 0.825
Epoch: 100, Learning Rate: 0.1
Epoch: 100 Accuracy: 0.825
Epoch: 150, Learning Rate: 0.1
Epoch: 150 Accuracy: 0.825
Epoch: 200, Learning Rate: 0.1
Epoch: 200 Accuracy: 0.825
Epoch: 250, Learning Rate: 0.1
Epoch: 250 Accuracy: 0.825
Epoch: 300, Learning Rate: 0.1
E

In [17]:
warnings.filterwarnings("ignore")
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
counter = 1
epochs = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
learning_rate_now=0.3

# Neural Network
for X in reduced_array:
    print(f"Set {counter}:")
    for epoch in epochs:
        print(f"Epoch: {epoch}, Learning Rate: {learning_rate_now}")
        array_accuracy_epoch = []
        for fold, (train_index, val_index) in enumerate(stratified_kfold.split(X, y)):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]
            mlp = MLPClassifier(
                hidden_layer_sizes=(15, 15),
                activation='relu',
                max_iter=epoch,
                random_state=42,
                learning_rate_init=learning_rate_now,
                alpha=0.1,
                beta_1=0.5,
                beta_2=0.5
            )
            mlp.out_activation_ = 'sigmoid'
            mlp.fit(X_train, y_train)
            y_val_pred = mlp.predict(X_val)
            # Evaluate the model's performance on the validation set
            accuracy_val = accuracy_score(y_val, y_val_pred)
            array_accuracy_epoch.append(accuracy_val)
        # Calculate and store the average accuracy across all folds
        avg_accuracy_epoch = np.max(array_accuracy_epoch)
        print(f"Epoch: {epoch} Accuracy: {avg_accuracy_epoch}")

    counter += 1





Set 1:
Epoch: 1, Learning Rate: 0.3
Epoch: 1 Accuracy: 0.5
Epoch: 50, Learning Rate: 0.3
Epoch: 50 Accuracy: 0.5641025641025641
Epoch: 100, Learning Rate: 0.3
Epoch: 100 Accuracy: 0.5641025641025641
Epoch: 150, Learning Rate: 0.3
Epoch: 150 Accuracy: 0.5641025641025641
Epoch: 200, Learning Rate: 0.3
Epoch: 200 Accuracy: 0.5641025641025641
Epoch: 250, Learning Rate: 0.3
Epoch: 250 Accuracy: 0.5641025641025641
Epoch: 300, Learning Rate: 0.3
Epoch: 300 Accuracy: 0.5641025641025641
Epoch: 350, Learning Rate: 0.3
Epoch: 350 Accuracy: 0.5641025641025641
Epoch: 400, Learning Rate: 0.3
Epoch: 400 Accuracy: 0.5641025641025641
Epoch: 450, Learning Rate: 0.3
Epoch: 450 Accuracy: 0.5641025641025641
Epoch: 500, Learning Rate: 0.3
Epoch: 500 Accuracy: 0.5641025641025641
Set 2:
Epoch: 1, Learning Rate: 0.3
Epoch: 1 Accuracy: 0.525
Epoch: 50, Learning Rate: 0.3
Epoch: 50 Accuracy: 0.525
Epoch: 100, Learning Rate: 0.3
Epoch: 100 Accuracy: 0.525
Epoch: 150, Learning Rate: 0.3
Epoch: 150 Accuracy: 0.525


In [18]:
warnings.filterwarnings("ignore")
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
counter = 1
epochs = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
learning_rate_now=0.5

# Neural Network
for X in reduced_array:
    print(f"Set {counter}:")
    for epoch in epochs:
        print(f"Epoch: {epoch}, Learning Rate: {learning_rate_now}")
        array_accuracy_epoch = []
        for fold, (train_index, val_index) in enumerate(stratified_kfold.split(X, y)):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]
            mlp = MLPClassifier(
                hidden_layer_sizes=(15, 15),
                activation='relu',
                max_iter=epoch,
                random_state=42,
                learning_rate_init=learning_rate_now,
                alpha=0.1,
                beta_1=0.5,
                beta_2=0.5
            )
            mlp.out_activation_ = 'sigmoid'
            mlp.fit(X_train, y_train)
            y_val_pred = mlp.predict(X_val)
            # Evaluate the model's performance on the validation set
            accuracy_val = accuracy_score(y_val, y_val_pred)
            array_accuracy_epoch.append(accuracy_val)
        # Calculate and store the average accuracy across all folds
        avg_accuracy_epoch = np.max(array_accuracy_epoch)
        print(f"Epoch: {epoch} Accuracy: {avg_accuracy_epoch}")

    counter += 1
            

Set 1:
Epoch: 1, Learning Rate: 0.5
Epoch: 1 Accuracy: 0.5
Epoch: 50, Learning Rate: 0.5
Epoch: 50 Accuracy: 0.5128205128205128
Epoch: 100, Learning Rate: 0.5
Epoch: 100 Accuracy: 0.5128205128205128
Epoch: 150, Learning Rate: 0.5
Epoch: 150 Accuracy: 0.5128205128205128
Epoch: 200, Learning Rate: 0.5
Epoch: 200 Accuracy: 0.5128205128205128
Epoch: 250, Learning Rate: 0.5
Epoch: 250 Accuracy: 0.5128205128205128
Epoch: 300, Learning Rate: 0.5
Epoch: 300 Accuracy: 0.5128205128205128
Epoch: 350, Learning Rate: 0.5
Epoch: 350 Accuracy: 0.5128205128205128
Epoch: 400, Learning Rate: 0.5
Epoch: 400 Accuracy: 0.5128205128205128
Epoch: 450, Learning Rate: 0.5
Epoch: 450 Accuracy: 0.5128205128205128
Epoch: 500, Learning Rate: 0.5
Epoch: 500 Accuracy: 0.5128205128205128
Set 2:
Epoch: 1, Learning Rate: 0.5
Epoch: 1 Accuracy: 0.525
Epoch: 50, Learning Rate: 0.5
Epoch: 50 Accuracy: 0.525
Epoch: 100, Learning Rate: 0.5
Epoch: 100 Accuracy: 0.525
Epoch: 150, Learning Rate: 0.5
Epoch: 150 Accuracy: 0.525


## Model 2

In [22]:
warnings.filterwarnings("ignore")
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
counter = 1
epochs = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
learning_rate_now=0.1

# Neural Network
for X in reduced_array:
    print(f"Set {counter}:")
    for epoch in epochs:
        print(f"Epoch: {epoch}, Learning Rate: {learning_rate_now}")
        array_accuracy_epoch = []
        for fold, (train_index, val_index) in enumerate(stratified_kfold.split(X, y)):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]
            mlp = MLPClassifier(
                hidden_layer_sizes=(20,20,20,20),
                activation='relu',
                max_iter=epoch,
                random_state=42,
                learning_rate_init=learning_rate_now,
                alpha=0.1,
                beta_1=0.5,
                beta_2=0.5
            )
            mlp.out_activation_ = 'sigmoid'
            mlp.fit(X_train, y_train)
            y_val_pred = mlp.predict(X_val)
            # Evaluate the model's performance on the validation set
            accuracy_val = accuracy_score(y_val, y_val_pred)
            array_accuracy_epoch.append(accuracy_val)
        # Calculate and store the average accuracy across all folds
        avg_accuracy_epoch = np.mean(array_accuracy_epoch)
        print(f"Epoch: {epoch} Accuracy: {avg_accuracy_epoch}")

    counter += 1





Set 1:
Epoch: 1, Learning Rate: 0.1
Epoch: 1 Accuracy: 0.5025000000000001
Epoch: 50, Learning Rate: 0.1
Epoch: 50 Accuracy: 0.5733333333333333
Epoch: 100, Learning Rate: 0.1
Epoch: 100 Accuracy: 0.5708333333333334
Epoch: 150, Learning Rate: 0.1
Epoch: 150 Accuracy: 0.5708333333333334
Epoch: 200, Learning Rate: 0.1
Epoch: 200 Accuracy: 0.5708333333333334
Epoch: 250, Learning Rate: 0.1
Epoch: 250 Accuracy: 0.5708333333333334
Epoch: 300, Learning Rate: 0.1
Epoch: 300 Accuracy: 0.5708333333333334
Epoch: 350, Learning Rate: 0.1
Epoch: 350 Accuracy: 0.5708333333333334
Epoch: 400, Learning Rate: 0.1
Epoch: 400 Accuracy: 0.5708333333333334
Epoch: 450, Learning Rate: 0.1
Epoch: 450 Accuracy: 0.5708333333333334
Epoch: 500, Learning Rate: 0.1
Epoch: 500 Accuracy: 0.5708333333333334
Set 2:
Epoch: 1, Learning Rate: 0.1
Epoch: 1 Accuracy: 0.49737179487179484
Epoch: 50, Learning Rate: 0.1
Epoch: 50 Accuracy: 0.593525641025641
Epoch: 100, Learning Rate: 0.1
Epoch: 100 Accuracy: 0.6037179487179488
Epoc

In [24]:
warnings.filterwarnings("ignore")
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
counter = 1
epochs = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
learning_rate_now=0.3

# Neural Network
for X in reduced_array:
    print(f"Set {counter}:")
    for epoch in epochs:
        print(f"Epoch: {epoch}, Learning Rate: {learning_rate_now}")
        array_accuracy_epoch = []
        for fold, (train_index, val_index) in enumerate(stratified_kfold.split(X, y)):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]
            mlp = MLPClassifier(
                hidden_layer_sizes=(20,20,20,20),
                activation='relu',
                max_iter=epoch,
                random_state=42,
                learning_rate_init=learning_rate_now,
                alpha=0.1,
                beta_1=0.5,
                beta_2=0.5
            )
            mlp.out_activation_ = 'sigmoid'
            mlp.fit(X_train, y_train)
            y_val_pred = mlp.predict(X_val)
            # Evaluate the model's performance on the validation set
            accuracy_val = accuracy_score(y_val, y_val_pred)
            array_accuracy_epoch.append(accuracy_val)
        # Calculate and store the average accuracy across all folds
        avg_accuracy_epoch = np.mean(array_accuracy_epoch)
        print(f"Epoch: {epoch} Accuracy: {avg_accuracy_epoch}")
    counter += 1





Set 1:
Epoch: 1, Learning Rate: 0.3
Epoch: 1 Accuracy: 0.525
Epoch: 50, Learning Rate: 0.3
Epoch: 50 Accuracy: 0.525
Epoch: 100, Learning Rate: 0.3
Epoch: 100 Accuracy: 0.525
Epoch: 150, Learning Rate: 0.3
Epoch: 150 Accuracy: 0.525
Epoch: 200, Learning Rate: 0.3
Epoch: 200 Accuracy: 0.525
Epoch: 250, Learning Rate: 0.3
Epoch: 250 Accuracy: 0.525
Epoch: 300, Learning Rate: 0.3
Epoch: 300 Accuracy: 0.525
Epoch: 350, Learning Rate: 0.3
Epoch: 350 Accuracy: 0.525
Epoch: 400, Learning Rate: 0.3
Epoch: 400 Accuracy: 0.525
Epoch: 450, Learning Rate: 0.3
Epoch: 450 Accuracy: 0.525
Epoch: 500, Learning Rate: 0.3
Epoch: 500 Accuracy: 0.525
Set 2:
Epoch: 1, Learning Rate: 0.3
Epoch: 1 Accuracy: 0.5
Epoch: 50, Learning Rate: 0.3
Epoch: 50 Accuracy: 0.525
Epoch: 100, Learning Rate: 0.3
Epoch: 100 Accuracy: 0.525
Epoch: 150, Learning Rate: 0.3
Epoch: 150 Accuracy: 0.525
Epoch: 200, Learning Rate: 0.3
Epoch: 200 Accuracy: 0.525
Epoch: 250, Learning Rate: 0.3
Epoch: 250 Accuracy: 0.525
Epoch: 300, Le

In [25]:
warnings.filterwarnings("ignore")
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
counter = 1
epochs = [1, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
learning_rate_now=0.5

# Neural Network
for X in reduced_array:
    print(f"Set {counter}:")
    for epoch in epochs:
        print(f"Epoch: {epoch}, Learning Rate: {learning_rate_now}")
        array_accuracy_epoch = []
        for fold, (train_index, val_index) in enumerate(stratified_kfold.split(X, y)):
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]
            mlp = MLPClassifier(
                hidden_layer_sizes=(20,20,20,20),
                activation='relu',
                max_iter=epoch,
                random_state=42,
                learning_rate_init=learning_rate_now,
                alpha=0.1,
                beta_1=0.5,
                beta_2=0.5
            )
            mlp.out_activation_ = 'sigmoid'
            mlp.fit(X_train, y_train)
            y_val_pred = mlp.predict(X_val)
            # Evaluate the model's performance on the validation set
            accuracy_val = accuracy_score(y_val, y_val_pred)
            array_accuracy_epoch.append(accuracy_val)
        # Calculate and store the average accuracy across all folds
        avg_accuracy_epoch = np.mean(array_accuracy_epoch)
        print(f"Epoch: {epoch} Accuracy: {avg_accuracy_epoch}")
    counter += 1





Set 1:
Epoch: 1, Learning Rate: 0.5
Epoch: 1 Accuracy: 0.5101282051282051
Epoch: 50, Learning Rate: 0.5
Epoch: 50 Accuracy: 0.5025641025641026
Epoch: 100, Learning Rate: 0.5
Epoch: 100 Accuracy: 0.5025641025641026
Epoch: 150, Learning Rate: 0.5
Epoch: 150 Accuracy: 0.5025641025641026
Epoch: 200, Learning Rate: 0.5
Epoch: 200 Accuracy: 0.5025641025641026
Epoch: 250, Learning Rate: 0.5
Epoch: 250 Accuracy: 0.5025641025641026
Epoch: 300, Learning Rate: 0.5
Epoch: 300 Accuracy: 0.5025641025641026
Epoch: 350, Learning Rate: 0.5
Epoch: 350 Accuracy: 0.5025641025641026
Epoch: 400, Learning Rate: 0.5
Epoch: 400 Accuracy: 0.5025641025641026
Epoch: 450, Learning Rate: 0.5
Epoch: 450 Accuracy: 0.5025641025641026
Epoch: 500, Learning Rate: 0.5
Epoch: 500 Accuracy: 0.5025641025641026
Set 2:
Epoch: 1, Learning Rate: 0.5
Epoch: 1 Accuracy: 0.4874358974358974
Epoch: 50, Learning Rate: 0.5
Epoch: 50 Accuracy: 0.49743589743589745
Epoch: 100, Learning Rate: 0.5
Epoch: 100 Accuracy: 0.49743589743589745
Ep

## Model 3